In [3]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
im_directory = '/Volumes/Tatum_SSD-1/Grad_School/m2py/Morphology_labels/OPV_morph_maps/3_component/'

im_filepaths = os.listdir(im_directory)
# print (im_filepaths)

files = []

for fl in im_filepaths:
    if fl[-1] == 'v':
        pass
    elif fl[-1] == 'e':
        pass
    else:
        files.append(fl)
        
print (files)


image_dict = {} # Hold all the image-like data
im_labels = {} # Gather all the processing conditions to be the predicted feature

for i,fl in enumerate(files):
    
    if fl[:-6] == files[i+1][:-6]:
        seg1 = np.load(im_directory+fl)
        seg2 = np.load(im_directory+files[i+1])
        
        sample = np.stack([seg1, seg1], axis = -1)
        
        print(sample.shape)
        
    else:
        pass
    
    image_dict[i] = sample
    
    anl_temp = 0
    anl_time = 0
    
    if 'NOANNEAL' in fl:
        pass
    
    else:
        temp_stop_indx = fl.index('C')
        anl_temp = int(fl[:temp_stop_indx])


        time_start_indx = temp_stop_indx+2
        time_stop_indx = fl.index('m')
        time_stop_indx = time_stop_indx
        anl_time = fl[time_start_indx:time_stop_indx]
        anl_time = int(anl_time)
        
    im_labels[i] = [anl_temp, anl_time]
    
im_labels_df = pd.fromDict(im_labels)
    
print (image_dict.shape)
im_labels.head()

['100C_15min_Sub4_Dev2_seg1.npy', '100C_15min_Sub4_Dev2_seg2.npy', '100C_15min_Sub4_Dev4_seg1.npy', '100C_15min_Sub4_Dev4_seg2.npy', '100C_15min_Sub4_Dev7_seg1.npy', '100C_15min_Sub4_Dev7_seg2.npy', '100C_30min_Sub7_Dev2_seg1.npy', '100C_30min_Sub7_Dev2_seg2.npy', '100C_30min_Sub7_Dev4_seg1.npy', '100C_30min_Sub7_Dev4_seg2.npy', '100C_30min_Sub7_Dev7_seg1.npy', '100C_30min_Sub7_Dev7_seg2.npy', '100C_5min_Sub1_Dev3_seg1.npy', '100C_5min_Sub1_Dev3_seg2.npy', '100C_5min_Sub1_Dev4_90D_seg1.npy', '100C_5min_Sub1_Dev4_90D_seg2.npy', '100C_5min_Sub1_Dev4_seg1.npy', '100C_5min_Sub1_Dev4_seg2.npy', '125C_15min_Sub4_Dev1_seg1.npy', '125C_15min_Sub4_Dev1_seg2.npy', '125C_15min_Sub4_Dev2_seg1.npy', '125C_15min_Sub4_Dev2_seg2.npy', '125C_15min_Sub4_Dev6_seg1.npy', '125C_15min_Sub4_Dev6_seg2.npy', '125C_15min_Sub4_Dev7_seg1.npy', '125C_15min_Sub4_Dev7_seg2.npy', '125C_15min_Sub4_postexam_seg1.npy', '125C_15min_Sub4_postexam_seg2.npy', '125C_30min_Sub8_Dev3_seg1.npy', '125C_30min_Sub8_Dev3_seg2.npy',

ValueError: substring not found

In [ ]:
X_im = image_dict #input features used to make prediction
Y_im = im_labels_df #target features to be predicted

x_im_train, x_im_test, y_im_train, y_im_test = train_test_split(X_im,Y_im, test_size = 0.2, shuffle = True) #split dataset into separate testing and training datasets

In [ ]:
# Hyper parameters
num_epochs = 10
im_batch_size = int(len(x_train[0])*0.9) # 90% of x_train samples
df_batch_size = int(len(x_train[0])*0.9) # 90% of x_train samples
im_learning_rate = 0.008
df_learning_rate = 0.001

# Device configuration (GPU if available, otherwise CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
x_im_train_tensor = torch.tensor(x_im_train.values.astype(np.float32)) #convert pd.DataFrame -> np.ndarray -> torch.tensor
y_im_train_tensor = torch.tensor(y_im_train.values.astype(np.float32))
im_train_tensor = torch.utils.data.TensorDataset(x_im_train_tensor, y_im_train_tensor) #create tensor with features and targets
im_training_data_set = torch.utils.data.DataLoader(dataset = im_train_tensor, batch_size = im_batch_size, shuffle = True) #create iterable dataset with batches

x_im_test_tensor = torch.tensor(x_im_test.values.astype(np.float32))
y_im_test_tensor = torch.tensor(yv_test.values.astype(np.float32))
im_test_tensor = torch.utils.data.TensorDataset(x_im_test_tensor, y_im_test_tensor)
im_testing_data_set = torch.utils.data.DataLoader(dataset = im_test_tensor, batch_size = im_batch_size, shuffle = True)


im_x, im_y, im_z = x_im[0].shape

In [ ]:
class ImBranchNN(nn.Module):
    
    def __init__(self, im_x, im_y, im_z):
        super(ImBranchNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2D(im_z, 32, kernal_size = 8, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2D(kernal_size = 2, stride = 2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2D(32, 64, kernal_size = 5, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2D(kernal_size = 2, stride = 2)
        )
            
            
        fc_nodes = (im_x/4) * (im_y/4) * 64 # 4 because there were 2 maxpool layers with 2x2 Kernals,
                                            # reducing the overall channel width & height by half each time
        
        self.layer3 = nn.Sequential(
            nn.dropout(),               #helps avoid over-fitting
            nn.Linear(fc_nodes, 10000), #Final kernal-size * im_x * im_y
            nn.Linear(10000, 5000),
            nn.Linear(5000, 1000),
            nn.Linear(1000, 100)
        )
        
        self.out_layer = nn.Linear(100, 2) #predicting anneal time and temp
            
    def forward(self, x_im):
        im_out = self.layer1(x_im)
        im_out = self.layer2(im_out)
        im_out = self.layer3(im_out)
        im_train_out = self.out_layer(im_out)
        
        return im_out, im_train_out

In [ ]:
def train_im_branch_one_epoch(model, im_training_data_set, im_criterion):
    total_step = len(im_training_data_set)
    loss_list = []
    
    model.train()

    for images, labels in im_training_data_set:
        # Run the forward pass
        im_out, im_train_out = model(images)
        optimizer.zero_grad()
        
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=im_learning_rate)
        
        # Gather the loss
        loss = im_criterion(outputs, labels)
        loss_list.append(loss.item())

        # backprop and perform Adam optimization
        torch.autograd.backward(loss)
        optimizer.step()
    
    total = len(loss_list)
    epoch_loss = sum(loss_list)/total
    
    return epoch_loss

In [ ]:
def eval_model(model, testing_data_set, criterion, optimizer):
    #evaluate the model
    model.eval()

    #don't update nodes during evaluation b/c not training
    with torch.no_grad():
        test_losses = []
        test_total = 0

        for inputs, labels in testing_data_set:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
    
            # calculate loss per batch of testing data
            test_loss = criterion(outputs, labels)
            test_losses.append(test_loss.item())
            test_total += 1

        total_test_loss = sum(test_losses)/test_total

        print (f"Total testing loss is: {total_test_loss}")
    return total_test_loss

In [ ]:
im_branch_model = ImBranchNN(im_x, im_y, im_z)

#define the loss function and the optimizer
im_criterion = nn.CrossEntropyLoss()
im_optimizer = torch.optim.Adam(model.parameters(), lr = im_learning_rate)

In [ ]:
im_train_epoch_losses = []
im_test_epoch_losses = []

for epoch in range(num_epochs):
    
    # First Train the image branch
    im_train_epoch_loss = train_im_branch_one_epoch(model = im_branch_model,
                                   training_data_set = im_training_data_set,
                                   criterion = im_criterion,
                                   optimizer = im_optimizer)
    
    im_train_epoch_losses.append(im_train_epoch_loss)
    
    im_test_epoch_loss = eval_im_branch(model = im_branch_model,
                                 testing_data_set = im_testing_data_set,
                                 criterion = im_criterion,
                                 optimizer = im_optimizer)
    
    im_test_epoch_losses.append(im_test_epoch_loss)

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

epochs = np.arange(1, (num_epochs+1), 1)

plt.plot(epochs, im_train_epoch_losses, c = 'k', label = 'training error')
plt.plot(epochs, im_test_epoch_losses, c = 'r', label = 'testing error')
plt.legend(loc = 'upper right')
plt.show()